# General Filter Neural Network (GFNN) の実装

 TENCON 2018で採択されているGFNNをKerasで実装した．
 論文( https://arxiv.org/pdf/1901.07375.pdf )

## Kerasの導入

In [1]:
import numpy as np

import tensorflow as tf
from keras import backend as K

Using TensorFlow backend.


In [0]:
from keras.models import Sequential
from keras.layers.convolutional import Conv2D
from keras.layers.pooling import MaxPooling2D
from keras.layers import ZeroPadding2D
from keras.optimizers import Adam

from keras.layers.core import Dense, Activation, Dropout, Flatten
from keras.utils import plot_model
from keras.callbacks import TensorBoard

from keras.datasets import mnist
from keras.utils import np_utils

## MNISTデータのインポート

In [0]:
num_classes = 10
img_height, img_width = 28, 28

In [0]:
(X_train, y_train),(X_test, y_test) = mnist.load_data()

# floatに型変換
X_train = X_train.astype('float32')[:, :, :, np.newaxis]
X_test = X_test.astype('float32')[:, :, :, np.newaxis]
# 各画素値を正規化
X_train /= 255.0
X_test /= 255.0

Y_train = np_utils.to_categorical(y_train, num_classes)
Y_test = np_utils.to_categorical(y_test, num_classes)

In [5]:
print(X_train.shape)

(60000, 28, 28, 1)


## CNNのモデルの実装

比較用のCNNモデルを実装．

In [6]:
cnn_model = Sequential()
    
cnn_model.add(Conv2D(41, (3, 3), padding='same', activation='relu', use_bias=False, input_shape=(img_height, img_width, 1)))
cnn_model.add(MaxPooling2D(pool_size=(2, 2)))
cnn_model.add(Conv2D(64, (3, 3), padding='same', activation='relu', use_bias=False))
cnn_model.add(MaxPooling2D(pool_size=(2, 2)))
cnn_model.add(Conv2D(128, (3, 3), padding='same', activation='relu', use_bias=False))
cnn_model.add(ZeroPadding2D(padding=(1, 1)))
cnn_model.add(MaxPooling2D(pool_size=(2, 2)))
    
cnn_model.add(Flatten())
cnn_model.add(Dense(625, activation='relu', use_bias=False))
cnn_model.add(Dropout(0.5))
cnn_model.add(Dense(num_classes, use_bias=False))
cnn_model.add(Activation('softmax'))

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [0]:
adam = Adam()
cnn_model.compile(optimizer=adam, loss='categorical_crossentropy', metrics=["accuracy"])

In [8]:
history = cnn_model.fit(X_train, Y_train, batch_size=512, epochs=10, verbose=1, validation_split=55000)

Instructions for updating:
Use tf.cast instead.
Epoch 1/10
60000/60000 [==============================] - 6s 105us/step - loss: 0.3674 - acc: 0.8841
Epoch 2/10
60000/60000 [==============================] - 4s 72us/step - loss: 0.0769 - acc: 0.9753
Epoch 3/10
60000/60000 [==============================] - 4s 72us/step - loss: 0.0517 - acc: 0.9839
Epoch 4/10
60000/60000 [==============================] - 4s 72us/step - loss: 0.0417 - acc: 0.9867
Epoch 5/10
60000/60000 [==============================] - 4s 72us/step - loss: 0.0321 - acc: 0.9898
Epoch 6/10
60000/60000 [==============================] - 4s 72us/step - loss: 0.0260 - acc: 0.9917
Epoch 7/10
60000/60000 [==============================] - 4s 72us/step - loss: 0.0243 - acc: 0.9921
Epoch 8/10
60000/60000 [==============================] - 4s 72us/step - loss: 0.0192 - acc: 0.9940
Epoch 9/10
60000/60000 [==============================] - 4s 72us/step - loss: 0.0193 - acc: 0.9938
Epoch 10/10
60000/60000 [==========================

In [9]:
cnn_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 28, 28, 41)        369       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 14, 14, 41)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 14, 14, 64)        23616     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 7, 7, 64)          0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 7, 7, 128)         73728     
_________________________________________________________________
zero_padding2d_1 (ZeroPaddin (None, 9, 9, 128)         0         
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 4, 4, 128)         0         
__________

In [10]:
cnn_score = cnn_model.evaluate(X_test, Y_test, verbose=0)
print('Test accuracy:', cnn_score[1])

Test accuracy: 0.9915


## GFNNの実装

### フィルタの設定
持ちいるフィルタに相当する畳み込みパラメータの重みを設定する．

鮮鋭化フィルタ（上下，斜め，8方向）

In [11]:
shapen = np.array([[[0, -1, 0], [-1, 5, -1], [0, -1, 0]],
                   [[-1, 0, -1], [0, 5, 0], [-1, 0, -1]],
                   [[-1, -1, -1], [-1, 9, -1], [-1, -1, -1]]])
print(shapen.shape)

(3, 3, 3)


エンボスフィルタ

In [12]:
emboss = np.array([[[-2, -1, 0], [-1, 1, 1], [0, 1, 2]]])
print(emboss.shape)

(1, 3, 3)


ブラーフィルタ（平滑化，ガウシアン）

In [13]:
blur1 = np.array([[[1., 1., 1.], [1., 1., 1.], [1., 1., 1.]]]) / 9.
blur2 = np.array([[[1., 2., 1.], [2., 4., 2.], [1., 2., 1.]]]) / 16.

blur = np.vstack((blur1, blur2))
print(blur.shape)

(2, 3, 3)


Robertsフィルタ

In [14]:
roberts = np.array([[[-0.6, -0.6, 1.], [-0.6, 0., 1.], [-0.6, -0.6, 1.]],
                   [[1., 1., 1.], [-0.6, 0., -0.6], [-0.6, -0.6, -0.6]],
                   [[1., -0.6, -0.6], [1., 0., -0.6], [1., -0.6, -0.6]],
                   [[-0.6, -0.6, -0.6], [-0.6, 0., -0.6], [1., 1., 1.]],
                   [[0.6, 0.6, -1.], [0.6, 0., -1.], [0.6, 0.6, -1.]],
                   [[-1., -1., -1.], [0.6, 0., 0.6], [0.6, 0.6, 0.6]],
                   [[-1., 0.6, 0.6], [-1., 0., 0.6], [-1., 0.6, 0.6]],
                   [[0.6, 0.6, 0.6], [0.6, 0., 0.6], [-1., -1., -1.]]])
print(roberts.shape)

(8, 3, 3)


Prewittフィルタ

In [15]:
prewitt = np.array([[[-1, -1, -1], [0, 0, 0], [1, 1, 1]], 
                   [[1, 1, 1], [0, 0, 0], [-1, -1, -1]],
                   [[-1, 0, 1], [-1, 0, 1], [-1, 0, 1]],
                   [[1, 0, -1], [1, 0, -1], [1, 0, -1]],
                   [[0, 1, 1], [-1, 0, 1], [-1, -1, 0]],
                   [[1, 1, 0], [1, 0, -1], [0, -1, -1]],
                   [[0, -1, -1], [1, 0, -1], [1, 1, 0]],
                   [[-1, -1, 0], [-1, 0, 1], [0, 1, 1]]])
print(prewitt.shape)

(8, 3, 3)


Sobelフィルタ

In [16]:
sobel =  np.array([[[-1, -2, -1], [0, 0, 0], [1, 2, 1]], 
                   [[1, 2, 1], [0, 0, 0], [-1, -2, -1]],
                   [[-1, 0, 1], [-2, 0, 2], [-1, 0, 1]],
                   [[1, 0, -1], [2, 0, -2], [1, 0, -1]],
                   [[0, 1, 2], [-1, 0, 1], [-2, -1, 0]],
                   [[2, 1, 0], [1, 0, -1], [0, -1, -2]],
                   [[0, -1, -2], [1, 0, -1], [2, 1, 0]],
                   [[-2, -1, 0], [-1, 0, 1], [0, 1, 2]]])
print(sobel.shape)

(8, 3, 3)


Frei-Chenフィルタ

In [17]:
frei_chen1 = np.array([[[-1., -np.sqrt(2), -1.], [0, 0, 0], [1., np.sqrt(2), 1.]]]) / (2. * np.sqrt(2))
frei_chen2 = np.array([[[1., np.sqrt(2), 1.], [0, 0, 0], [-1., -np.sqrt(2), -1.]]]) / (2. * np.sqrt(2))
frei_chen3 = np.array([[[-1., 0, 1.], [-np.sqrt(2), 0, np.sqrt(2)], [-1., 0, 1.]]]) / (2. * np.sqrt(2))
frei_chen4 = np.array([[[1., 0, -1.], [np.sqrt(2), 0, -np.sqrt(2)], [1., 0, -1.]]]) / (2. * np.sqrt(2))

frei_chen5 = np.array([[[0, 1., 0], [-1., 0, -1.], [0, 1., 0]]]) / 2.
frei_chen6 = np.array([[[-1., 0, 1.], [0, 0, 0], [1., 0, -1.]]]) / 2.

frei_chen7 = np.array([[[1., -2., 1.], [-2., 4., -2.], [1., -2., 1.]]]) / 6.
frei_chen8 = np.array([[[-2., 1., -2.], [1., 4., 1.], [-2., 1., -2.]]]) / 6.

frei_chen = np.vstack((frei_chen1, frei_chen2, frei_chen3, frei_chen4, frei_chen5, frei_chen6, frei_chen7, frei_chen8))
print(frei_chen.shape)

(8, 3, 3)


2階微分フィルタ

In [18]:
laplacian = np.array([[[0 ,1, 0], [1, -4, 1], [0, 1, 0]], [[1, 1, 1], [1, -8, 1],[1, 1, 1]]])
print(laplacian.shape)

(2, 3, 3)


離散コサイン変換(DCT)フィルタ

In [19]:
import math
dct = np.array([[[np.sqrt(0.5), np.sqrt(0.5), np.sqrt(0.5)],
                [np.cos((1.  * math.pi ) / 6.), np.cos((3.  * math.pi ) / 6.), np.cos((5.  * math.pi ) / 6.)],
                [np.cos((2.  * math.pi ) / 6.), np.cos((6.  * math.pi ) / 6.), np.cos((10.  * math.pi ) / 6.)]]]) * np.sqrt(1. / 2.)
print(dct.shape)

(1, 3, 3)


フィルタ全結合

In [20]:
img_filter = np.vstack((shapen, emboss, blur, roberts, prewitt, sobel, frei_chen, laplacian, dct)).astype(np.float32)
print(img_filter.shape)

(41, 3, 3)


### GFNNモデルの実装
次にGFNNのモデルを実装する

In [0]:
gfnn_model = Sequential()

gfnn_model.add(Conv2D(41, (3, 3), padding='same', activation='relu', use_bias=False, input_shape=(img_height, img_width, 1), trainable=False))
gfnn_model.add(MaxPooling2D(pool_size=(2, 2)))
gfnn_model.add(Conv2D(64, (3, 3), padding='same', activation='relu', use_bias=False))
gfnn_model.add(MaxPooling2D(pool_size=(2, 2)))
gfnn_model.add(Conv2D(128, (3, 3), padding='same', activation='relu', use_bias=False))
gfnn_model.add(ZeroPadding2D(padding=(1, 1)))
gfnn_model.add(MaxPooling2D(pool_size=(2, 2)))
    
gfnn_model.add(Flatten())
gfnn_model.add(Dense(625, activation='relu', use_bias=False))
gfnn_model.add(Dropout(0.5))
gfnn_model.add(Dense(num_classes, use_bias=False))
gfnn_model.add(Activation('softmax'))

In [0]:
adam = Adam()
gfnn_model.compile(optimizer=adam, loss='categorical_crossentropy', metrics=["accuracy"])

In [23]:
gfnn_model.summary()
weight = gfnn_model.get_weights()
weight[0] = img_filter[:, :, :, np.newaxis].transpose((1, 2, 3, 0))
gfnn_model.set_weights(weight)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_4 (Conv2D)            (None, 28, 28, 41)        369       
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 14, 14, 41)        0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 14, 14, 64)        23616     
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 7, 7, 64)          0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 7, 7, 128)         73728     
_________________________________________________________________
zero_padding2d_2 (ZeroPaddin (None, 9, 9, 128)         0         
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 4, 4, 128)         0         
__________

In [24]:
gfnn_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_4 (Conv2D)            (None, 28, 28, 41)        369       
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 14, 14, 41)        0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 14, 14, 64)        23616     
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 7, 7, 64)          0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 7, 7, 128)         73728     
_________________________________________________________________
zero_padding2d_2 (ZeroPaddin (None, 9, 9, 128)         0         
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 4, 4, 128)         0         
__________

In [25]:
history = gfnn_model.fit(X_train, Y_train, batch_size=512, epochs=10, verbose=1, validation_split=55000)

Epoch 1/10
60000/60000 [==============================] - 4s 60us/step - loss: 0.2965 - acc: 0.9102
Epoch 2/10
60000/60000 [==============================] - 3s 55us/step - loss: 0.0545 - acc: 0.9828
Epoch 3/10
60000/60000 [==============================] - 3s 55us/step - loss: 0.0371 - acc: 0.9887
Epoch 4/10
60000/60000 [==============================] - 3s 55us/step - loss: 0.0285 - acc: 0.9908
Epoch 5/10
60000/60000 [==============================] - 3s 55us/step - loss: 0.0210 - acc: 0.9937
Epoch 6/10
60000/60000 [==============================] - 3s 55us/step - loss: 0.0178 - acc: 0.9946
Epoch 7/10
60000/60000 [==============================] - 3s 56us/step - loss: 0.0197 - acc: 0.9939
Epoch 8/10
60000/60000 [==============================] - 3s 55us/step - loss: 0.0126 - acc: 0.9961
Epoch 9/10
60000/60000 [==============================] - 3s 55us/step - loss: 0.0124 - acc: 0.9959
Epoch 10/10
60000/60000 [==============================] - 3s 56us/step - loss: 0.0097 - acc: 0.9966

In [26]:
gfnn_score = gfnn_model.evaluate(X_test, Y_test, verbose=0)
print('Test accuracy:', gfnn_score[1])

Test accuracy: 0.9936
